# Merging data

In [1]:
import pandas as pd
import numpy as np

In [4]:
# Read data 

# opioid
opioid = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/opi_merge_final.parquet", engine='fastparquet')

# vital deaths
vital = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/Vital.parquet", engine='fastparquet')

# population
population = pd.read_parquet("/Users/preetkhowaja/Documents/MIDS Fall 2021/Practicing DS/pds2021-opioids-pds6/10_modified_data/pop_final.parquet", engine='fastparquet')

In [5]:
opioid.sample(10)

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME
index,,,,,,,
5976,BAMBERG,SC,45009,SC,BAMBERG,2012,5.189054e+06
3971,OKTIBBEHA,MS,28105,MS,OKTIBBEHA,2014,9.319926e+06
5995,BEAUFORT,SC,45013,SC,BEAUFORT,2013,3.764649e+07
5585,WAGONER,OK,40145,OK,WAGONER,2014,4.084806e+07
8705,WALLA WALLA,WA,53071,WA,WALLA WALLA,2011,2.074217e+07
1829,HEARD,GA,13149,GA,HEARD,2007,1.358462e+06
7541,LLANO,TX,48299,TX,LLANO,2012,5.509634e+06
1601,DOUGHERTY,GA,13095,GA,DOUGHERTY,2007,1.606992e+07
1635,EFFINGHAM,GA,13103,GA,EFFINGHAM,2014,1.683248e+07


In [6]:
vital.sample(10)

,County,County Code,Year,Deaths
7303,"Wabash County, IN",18169,2008,10
1888,"Dearborn County, IN",18029,2012,11
2937,"Hawkins County, TN",47073,2009,12
757,"Brunswick County, NC",37019,2003,10
2123,"Eddy County, NM",35015,2010,16
7844,"Yamhill County, OR",41071,2015,10
6958,"Sussex County, DE",10005,2009,27
1763,"Dakota County, MN",27037,2009,27
295,"Baltimore city, MD",24510,2005,231
6854,"Steuben County, NY",36101,2007,10


In [7]:
population.sample(10)

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,FIPS
532,13,273,Georgia,Terrell County,2006,9969,13,273,13273
439,13,85,Georgia,Dawson County,2006,20660,13,085,13085
15070,42,49,Pennsylvania,Erie County,2010,280566,42,049,42049
1186,22,111,Louisiana,Union Parish,2006,22919,22,111,22111
3961,18,117,Indiana,Orange County,2007,19868,18,117,18117
14200,28,23,Mississippi,Clarke County,2010,16732,28,023,28023
26699,22,81,Louisiana,Red River Parish,2014,8757,22,081,22081
11336,31,165,Nebraska,Sioux County,2009,1324,31,165,31165
13790,21,31,Kentucky,Butler County,2010,12690,21,031,21031
24435,39,41,Ohio,Delaware County,2013,185380,39,041,39041


In [8]:
population[population.duplicated(["FIPS", "Year"], keep=False)==True]

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,FIPS
325,11,0,District of Columbia,District of Columbia,2006,570681,11,000,11000
326,11,0,District of Columbia,District of Columbia,2006,570681,11,000,11000
327,11,1,District of Columbia,District of Columbia,2006,570681,11,001,11001
328,11,1,District of Columbia,District of Columbia,2006,570681,11,001,11001
3516,11,0,District of Columbia,District of Columbia,2007,574404,11,000,11000
3517,11,0,District of Columbia,District of Columbia,2007,574404,11,000,11000
3518,11,1,District of Columbia,District of Columbia,2007,574404,11,001,11001
3519,11,1,District of Columbia,District of Columbia,2007,574404,11,001,11001
6707,11,0,District of Columbia,District of Columbia,2008,580236,11,000,11000
6708,11,0,District of Columbia,District of Columbia,2008,580236,11,000,11000


In [9]:
# Merge opioid and vital first

merge1 = pd.merge(opioid, vital, left_on=["fips", "year"], right_on=["County Code", "Year"], how='inner')

merge1

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,MME,County,County Code,Year,Deaths
0,BALDWIN,AL,1003,AL,BALDWIN,2006,5.103983e+07,"Baldwin County, AL",1003,2006,11
1,BALDWIN,AL,1003,AL,BALDWIN,2007,6.032305e+07,"Baldwin County, AL",1003,2007,24
2,BALDWIN,AL,1003,AL,BALDWIN,2008,6.682852e+07,"Baldwin County, AL",1003,2008,19
3,BALDWIN,AL,1003,AL,BALDWIN,2009,7.638538e+07,"Baldwin County, AL",1003,2009,29
4,BALDWIN,AL,1003,AL,BALDWIN,2010,9.272014e+07,"Baldwin County, AL",1003,2010,24
...,...,...,...,...,...,...,...,...,...,...,...
1775,YAKIMA,WA,53077,WA,YAKIMA,2010,6.746887e+07,"Yakima County, WA",53077,2010,24
1776,YAKIMA,WA,53077,WA,YAKIMA,2011,7.600735e+07,"Yakima County, WA",53077,2011,14
1777,YAKIMA,WA,53077,WA,YAKIMA,2012,8.023265e+07,"Yakima County, WA",53077,2012,25
1778,YAKIMA,WA,53077,WA,YAKIMA,2013,8.391631e+07,"Yakima County, WA",53077,2013,14


In [10]:
# Convert the fips column in merge1 data to strings and fill them to 5 digits. (for better merging with population data)

merge1["fips"] = merge1["fips"].astype(str)

merge1["fips"] = merge1["fips"].apply(lambda x: x.zfill(5))

merge1 = merge1.drop(columns=["year", "County Code"])

# Convert the year column in population data to int64 so that the type matches

population["Year"] = population["Year"].astype(int)

In [11]:
# Merge opioid and vital deaths data with population data, then

merge2 = pd.merge(merge1, population, left_on=["fips", "Year"], right_on=["FIPS", "Year"], how='inner')

merge_final = merge2[["BUYER_STATE", "BUYER_COUNTY", "Year", "MME", "Deaths", "Population", "fips"]]

merge_final["BUYER_STATE"].value_counts()


FL    319
TX    314
GA    232
LA    141
WA    140
AL    133
SC    133
OK    112
NM     86
OR     71
MS     46
ID     31
MT     22
Name: BUYER_STATE, dtype: int64

In [15]:
## Checking if we have same number of rows for all years. 
merge_final['Year'].value_counts()



2014    216
2013    211
2011    207
2012    207
2010    203
2009    198
2008    193
2007    177
2006    168
Name: Year, dtype: int64